# NLP Doc2Vec + Word2vec (skipgram) Model for Recommending Arabic Text

### After we extracted the data. We have to train our model using it.

# Importing Necessary Modules.

In [1]:
import pandas as pd
import gensim as gs
import multiprocessing
from gensim.models.doc2vec import Doc2Vec

C:\Users\dell\Anaconda3\lib\site-packages\gensim\utils.py:862: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


# Loading up the wikipedia data that extracted into a dataframe.

Just to mention: Here we are loading up a part of the data we extracted (about 700MB)

In [2]:
cleaned_articles_df_path = "F:\\Hackathon\\easyLearn\\cleaner 1 gb\\cleaner_one_gb_articles.csv"

In [3]:
articles = pd.read_csv(cleaned_articles_df_path,encoding='utf-16')

In [4]:
columns = ['id','title','text']
articles = articles[columns]
articles.head()

,id,title,text
0,7,ماء,الاكثر شيوعا في الارض غاز بخار ما سحاب الما ه...
1,8,ويكيبيديا:How to edit Arabic pages,تصنيف مساعده ويكيبيديا اسم الصفحه
2,30,ويكيبيديا:الميدان,محميه فهرسمجتمع ميدان رجا لا تعدل هذه الصفحه ...
3,38,رياضيات,في القرن الثالث كما تخيله رفائيل في لوحته الم...
4,46,الصفحة الرئيسية,لافهرس لاتحريرقسم مقدمه الصفحه الرئيسيه عنوان...


## Let's drop any Nan values (if there is any)

In [5]:
articles.dropna()

,id,title,text
0,7,ماء,الاكثر شيوعا في الارض غاز بخار ما سحاب الما ه...
1,8,ويكيبيديا:How to edit Arabic pages,تصنيف مساعده ويكيبيديا اسم الصفحه
2,30,ويكيبيديا:الميدان,محميه فهرسمجتمع ميدان رجا لا تعدل هذه الصفحه ...
3,38,رياضيات,في القرن الثالث كما تخيله رفائيل في لوحته الم...
4,46,الصفحة الرئيسية,لافهرس لاتحريرقسم مقدمه الصفحه الرئيسيه عنوان...
5,62,إستونيا,ترجمه اليه تاريخ يوليو صم بلد اسم الدوله جمهو...
6,71,سنجاب,مصدر تاريخ يناير صندوق معلومات كائن ماهيه الت...
7,76,تلفاز,التلفاز وجمعه تلافز وتلفازات او التلفزه او ال...
8,77,A,ويكاموس حرف اسم همزهالف مفخمه رسم كتابه لاتين...
9,78,ويكيبيديا:بلاغ أخطاء وطلب ميزات,اختصار وب وب وب بلاغ الاخطا وطلب الميزات الجد...


## Again, we're gonna to make sure that all values under the text column are of type string.

In [7]:
articles['text'] = articles['text'].astype(str)
articles['text']

0          الاكثر شيوعا في الارض غاز بخار ما سحاب الما ه...
1                         تصنيف مساعده ويكيبيديا اسم الصفحه
2          محميه فهرسمجتمع ميدان رجا لا تعدل هذه الصفحه ...
3          في القرن الثالث كما تخيله رفائيل في لوحته الم...
4          لافهرس لاتحريرقسم مقدمه الصفحه الرئيسيه عنوان...
5          ترجمه اليه تاريخ يوليو صم بلد اسم الدوله جمهو...
6          مصدر تاريخ يناير صندوق معلومات كائن ماهيه الت...
7          التلفاز وجمعه تلافز وتلفازات او التلفزه او ال...
8          ويكاموس حرف اسم همزهالف مفخمه رسم كتابه لاتين...
9          اختصار وب وب وب بلاغ الاخطا وطلب الميزات الجد...
10         مصدر تاريخ يناير حرفلاتيني اف هو الحرف السادس...
11         مصدر تاريخ يناير حرفلاتيني كي هو الحرف الحادي...
12         صندوق معلومات كائن المجال الزمني الاحفوري الب...
13         من السهل جدا تحرير وتعديل اغلب صفحات الموسوعه...
14         رساله توضيح مدينه قرطاج الحديثه تونس بالجمهور...
15         وضح السنافر توضيح معلومات تلفاز اسم السنافر ص...
16          بوابه احداث جاريهمتصفح الاخب

# Let's build the model with the help of gensim library.

## Gensim is a useful library when you want to do deep learning and natural language processing with text using python.

### You can find out more about it here: https://radimrehurek.com/gensim/intro.html

## Generating an Iterable of TaggedDocument Objects

In [11]:
%%time
documents = []
for index,row in articles.iterrows():
     documents.append(gs.models.doc2vec.TaggedDocument(words=row['text'].split(),tags=['SENT_%s' % row['id']]))

Wall time: 30.6 s


### Getting number of cores of my machine's cpu

In [12]:
cores = multiprocessing.cpu_count()

## Initializing the Doc2Vec model

### Description of the parameters we used in the Doc2Vec model initialization function

#### Parameters:
1. **dm** (int {1,0}) – Defines the training algorithm. If dm=1, ‘distributed memory’ (PV-DM) is used. Otherwise, distributed bag of words (PV-DBOW) is employed.
2. **dbow_words** (int {1,0}) – If set to 1 trains word-vectors (in skip-gram fashion) simultaneous with DBOW doc-vector training; If 0, only trains doc-vectors (faster).
3. **size** (int) – Dimensionality of the feature vectors.
4. **window** (int) – The maximum distance between the current and predicted word within a sentence.
5. **min_count** (int) – Ignores all words with total frequency lower than this.
6. **workers** (int) – Use these many worker threads to train the model (=faster training with multicore machines).

#### You can read more about parameters in the documentation of the gensim library: https://radimrehurek.com/gensim/models/doc2vec.html 

### Some Notes about choosing the values of the Doc2Vec parameters:
When you increase the value of the window, the duration of training is going to be longer (but usually with better results).
Not just **window** can affect the duration of training. Also, the **size** can obviously have its effect on the duration of training.
You have also not to forget the more you increase those values the more computational power you they are going to need.

In [13]:
%%time
# PV-DBOW 
model = Doc2Vec(dm=0, dbow_words=1, size=500, window=20, min_count=5, workers=cores)

Wall time: 145 ms


## Building the vocabs of the model using the list 'documents' that we created earlier.
### When you build your model your going to need this is step.
#### Here we are passing the documents list as a parameter. 
#### In general, The documents iterable can be simply a list of TaggedDocument elements, but for larger corpora, consider an iterable that streams the documents directly from disk/network. 
You can find out more about this in the documentation.

In [14]:
%%time
model.build_vocab(documents)

Wall time: 47.5 s


# Training the Model.

## Description of the parameters we used in the Doc2Vec.train() function

#### Parameters:
1. **documents** 
2. **total_words** (int) – Count of raw words in documents.
3. **epochs** (int) – Number of iterations (epochs) over the corpus.

### Some notes about choosing the values of the Doc2Vec.train() parameters

The **documents** is the corpus (the larger the corpus the longer it takes to train).
When you increase the value **total_words**, you will hopefully get a better results but it's going to need more time to train.
The **epochs** parameters have a big effect on the duration of training process which is obvious. So you can start with low epochs and see what you get. But before deciding how much epochs your going to choose you **have to consider** the parameters that we used earlier when we initialized our model especially the **window** parameter.
So you can realize that if you'd like to choose high values of the parameters, basically you are going to have better results (hopefully) but on the expense of time and computational power (Nothing comes without a cost) :)

In [15]:
%%time
model.train(documents, total_words=100000,epochs=3)

Wall time: 1h 32min 55s


159855707

In [16]:
str(model)

'Doc2Vec(dbow+w,d500,n5,w20,mc5,s0.001,t4)'

## After we finished training we can now take a sneak peak at the results.

### Now, we can get a dataframe representation of the documents list and save it to my hard drive (optional)

In [17]:
temp_df = pd.DataFrame(documents)

In [18]:
temp_df.head()

,words,tags
0,"[الاكثر, شيوعا, في, الارض, غاز, بخار, ما, سحاب...",[SENT_7]
1,"[تصنيف, مساعده, ويكيبيديا, اسم, الصفحه]",[SENT_8]
2,"[محميه, فهرسمجتمع, ميدان, رجا, لا, تعدل, هذه, ...",[SENT_30]
3,"[في, القرن, الثالث, كما, تخيله, رفائيل, في, لو...",[SENT_38]
4,"[لافهرس, لاتحريرقسم, مقدمه, الصفحه, الرئيسيه, ...",[SENT_46]


### Here we are converting the type of the values under the tags column from list to string (just to make it easier to deal with)

In [19]:
convert_list_to_str = lambda lis: lis[0]
temp_df['tags'] = temp_df['tags'].apply(convert_list_to_str)
temp_df.head()

,words,tags
0,"[الاكثر, شيوعا, في, الارض, غاز, بخار, ما, سحاب...",SENT_7
1,"[تصنيف, مساعده, ويكيبيديا, اسم, الصفحه]",SENT_8
2,"[محميه, فهرسمجتمع, ميدان, رجا, لا, تعدل, هذه, ...",SENT_30
3,"[في, القرن, الثالث, كما, تخيله, رفائيل, في, لو...",SENT_38
4,"[لافهرس, لاتحريرقسم, مقدمه, الصفحه, الرئيسيه, ...",SENT_46


## Let's define some functions that will help us in viewing the recommender results in a better fashion.

In [25]:
def get_titles(sim_list, art_dataframe):
    titles = []
    for (tag,_) in sim_list:
        tag_id = int(tag.split('_')[1])
        titles.append(art_dataframe[art_dataframe['id'] == tag_id]['title'].values[0])
    return titles

In [20]:
def concatenate_list_data(list):
    result= ''
    for element in list:
        result += ' ' + str(element)
    return result

In [26]:
def get_texts(sim_list, art_dataframe):
    texts = []
    for (tag,_) in sim_list:
        tag_id = int(tag.split('_')[1])
        texts.append(art_dataframe[art_dataframe['id'] == tag_id]['text'].values[0])
    return texts

### 1st example

### Let's take a look at an article that talks about water in Arabic and see what we can get with the recommender.

In [22]:
# the water article
water = articles[articles['id'] == 7]['text'][0]

In [23]:
water

' الاكثر شيوعا في الارض غاز بخار ما سحاب الما هو سائل شفاف لا لون له ولا رائحه ويوجد في الكره الارضيه في المسطحات المائيه من الجداول والبحيرات والبحار والمحيطات او يتساقط عليها علي شكل مطر امطار كما يعد المكون الاساسي للسوائل في جميع الكائنات الحيه ويعد الما اكثر مركب كيميائي المركبات الكيميائيه انتشارا ويتالف خواص الما جزي الما من ثلاث ذرات ذره اكسجين مركزيه ترتبط بها ذرتي هيدروجين رابطه تساهميه برابطه تساهميه لتكون الصيغه الكيميائيه صيغته يكون الما سائل سائلا عند الظروف القياسيه من الضغط ودرجه الحراره ولكن له حالات الماده حالات اخري شائعه الوجود ايضا وهي الحاله الصلبه علي شكل جليد والحاله الغازيه علي شكل بخار ما ان الما هو اساس وجود الحياه علي كوكب الارض وهو يغطي من سطحها وتمثل مياه البحار والمحيطات اكبر نسبه للما علي الارض حيث تبلغ حوالي وتتوزع النسب الباقيه بين مياه جوفيه المياه الجوفيه وبين مثلجه جليد المناطق القطبيه لكليهما مع وجود نسبه صغيره علي شكل بخار ما معلق في الهوا علي هيئه سحاب غيوم واحيانا اخري علي هيئه ضباب او ندي بالاضافه الي هطول الزخات مطر المطريه او ثلج الثلجيه مرجع

In [24]:
tokens = water.split()

new_vector = model.infer_vector(tokens)
sims = model.docvecs.most_similar([new_vector]) #gives you top 10 document tags and their cosine similarity

sims

[('SENT_645007', 0.6346936225891113),
 ('SENT_7', 0.6192128658294678),
 ('SENT_2793', 0.6158888339996338),
 ('SENT_301167', 0.6148782968521118),
 ('SENT_1250', 0.611820638179779),
 ('SENT_112193', 0.6112213730812073),
 ('SENT_1249', 0.6110888123512268),
 ('SENT_97114', 0.6087453365325928),
 ('SENT_251004', 0.6084399819374084),
 ('SENT_167988', 0.6084308624267578)]

### Getting the titles of all resulted articles. 

In [27]:
get_titles(sims,articles)

['تسمم بالرصاص',
 'ماء',
 'الأرض',
 'اضطراب نقص الانتباه مع فرط النشاط',
 'ابن خلدون',
 'دب قطبي',
 'عمان (مدينة)',
 'سمنة',
 'ارتفاع ضغط الدم',
 'حصاة كلوية']

### Getting the text of all resulted articles. 

In [28]:
get_texts(sims,articles)

[' معلومات مرض صوره قبالامراض تدا تدا تدا تدا تداا وراثه مندليه بشريه مدلاين بلس ماده اي ميديسين موضوع اي ميديسين معرف نفمط معرف مراجعه جين اسم مراجعه جين التسمم رصاص بالرصاص هو نوع من انواع التسمم معدن المعدني الناجم عن تراكم رصاص الرصاص في الجسم يعتبر دماغ الدماغ هو اكثر الاعضا حساسيه للرصاص مرجع ويب العنوان المسار الموقع تاريخ الوصول التاريخ قد تشمل اعراض التسمم الم بطني الام في البطن امساك دا والامساك صداع والصداع تهيجيه والتهيج ومشاكل الذاكره وعدم القدره علي انجاب الاطفال عقم العقم وخز في اليدين والقدمين يسبب التسمم بالرصاص ما يقرب من من حالات تخلف عقلي التخلف العقلي التي تحدث لاسباب غير معروفه وخلافا لذلك ويمكن ان يؤدي ذلك الي مشاكل سلوكيه يخلف تسمم الرصاص بعض الاثار الدائمه مرجع ويب العنوان المسار الموقع تاريخ الوصول التاريخ وقد يحدث في الحالات الشديده فقر الدم انيميا فقر الدم نوبه طب والنوبات غيبوبه والغيبوبه او الوفاه يمكن ان يتعرض المر للرصاص عن طريق الهوا الملوث او الما او الغبار او الطعام او المنتجات الاستهلاكيه الاطفال معرضون لخطر اكبر لانهم اكثر عرضه لالتقاط الاشيا من الا

### example 2

### This is bascially a wikipedia article that talks about mathematics.

In [29]:
math = articles[articles['id'] == 38]['text'][3]

In [30]:
math

' في القرن الثالث كما تخيله رفائيل في لوحته المعروفه مدرسه اثينا لوحه مدرسه اثينا الرياضيات باليونانيه وترجمها كل من ابن رشد و الي كلمتي التعاليميه والتعليميه علم عباره عن مفاهيم مجرده واصطلاحات رياضيه تدل علي الكم و العدد يدل علي كميه المعدود والمقدار قابل للزياده او النقصان وعندما نستطيع قياس المقدار نطلق عليه اسم الكم لذلك عرف بعض عالم مهنه العلما الرياضيات بانه علم القياس وتعتبر الرياضيات لغه العلوم اذ ان هذه العلوم لا تكتمل الا عندما نحول نتائجها الي معادله معادلات ونحول ثوابتها الي خطوط بيانيه تعرف الرياضيات بانها دراسه القياس و الحساب و الهندسه هذا بالاضافه الي المفاهيم الحديثه نسبيا ومنها البنيه الفضا او الفراغ و التغير و بعد الابعاد وبشكل عام قد يعرفها البعض علي انها دراسه البني المجرده باستخدام المنطق و برهان رياضي البراهين الرياضيه و تدوين رياضي التدوين الرياضي وبشكل اكثر عموميه قد تعرف الرياضيات ايضا علي انها دراسه الاعداد وانماطها ولقد نشا علم الرياضيات عندما قاس انسان الانسان ما شاهده من ظواهر طبيعه طبيعيه وبنا علي فطره وخاصيه في انسان الانسان الا وهي اهتمامه بقياس كل ما 

In [31]:
tokens2 = math.split()

new_vector2 = model.infer_vector(tokens2)
sims2 = model.docvecs.most_similar([new_vector2]) #gives you top 10 document tags and their cosine similarity

sims2

[('SENT_173260', 0.5185093879699707),
 ('SENT_83203', 0.5132399797439575),
 ('SENT_81020', 0.5107713937759399),
 ('SENT_136650', 0.5102975368499756),
 ('SENT_315054', 0.5074890851974487),
 ('SENT_38', 0.5060461759567261),
 ('SENT_81016', 0.5052069425582886),
 ('SENT_729968', 0.5047352910041809),
 ('SENT_676767', 0.5043123960494995),
 ('SENT_410157', 0.5036652088165283)]

In [32]:
get_titles(sims2,articles)

['بوابة:علوم/بوابات شقيقة',
 'بوابة:علم الأحياء/فروع علم الأحياء',
 'حقول العلم',
 'بوابة:علوم/علوم طبيعية',
 'قائمة الدول حسب السكان المهاجرين إليها',
 'رياضيات',
 'قائمة التخصصات الأكاديمية',
 'علم الأحياء الاجتماعي',
 'ويكيبيديا:مقالات مطلوبة حسب الاختصاص/علم الأحياء',
 'أحيائي']

In [33]:
get_texts(sims2,articles)

['  بوابه تاريخ العلوم تاريخ العلوم بوابه فلسفه العلوم فلسفه العلوم العلوم الطبيعيه والرياضيات علوم احيائيه بوابه علم الاحيا علم الاحيا بوابه زراعه زراعه بوابه علم الحيوان علم الحيوان بوابه مفصليات مفصليات بوابه زواحف زواحف بوابه تقانه حيويه تقانه حيويه بوابه طيور طيور بوابه سنوريات سنوريات بوابه حيتانيات حيتانيات بوابه ديناصورات ديناصورات بوابه كلبيات كلبيات بوابه انقراض انقراض بوابه علم الاحيا التطوري علم الاحيا التطوري بوابه عالم بحري عالم بحري بوابه فطريات فطريات بوابه ثديات ثديات بوابه العقل والدماغ العقل والدماغ بوابه علم الاحيا الخلوي والجزيئي علم الاحيا الخلوي والجزيئي بوابه علوم عصبيه علوم عصبيه بوابه علم النبات علم النبات بوابه قروش قروش علوم صحيه بوابه طب طب بوابه طب اسنان طب اسنان بوابه صيدله صيدله علوم فيزيائيه بوابه علم الفلك علم الفلك بوابه تقويم تقويم بوابه الكيميا الكيميا بوابه علم البيئه علم البيئه بوابه طاقه طاقه بوابه علوم الارض علوم الارض بوابه براكين براكين بوابه طقس طقس بوابه الفيزيا الفيزيا بوابه ما ما بوابه علم الكون علم الكون بوابه رياضيات رياضيات بوابه تعميه 

### example 3

### This is an article that talks about operating systems.

In [35]:
operating_sys = articles[articles['id'] == 1275]['text'][29]

In [36]:
operating_sys

' انظمه تشغيل نظام التشغيل انج وتختصر الي هو مجموعه من البرمجيات المسؤوله عن اداره الموارد عتاد الحاسوب و برمجيه برمجيات حاسوب الحاسوب ويمثل وسيط بين المستخدم و عتاد الحاسوب وبتعريف اخر يمثل نظام التشغيل جسر لتشغيل برامج المستخدم ويقوم بالمهام الاساسيه مثل اداره وتخصيص مصادر الحاسوب الذاكره القرص الصلب الوصول للاجهزه الملحقه الخ وترتيب اولويه التعامل مع الاوامر والتحكم في اجهزه الادخال والاخراج مثل لوحه المفاتيح وكذلك لتسهيل التعامل مع الشبكات واداره الملفات مقدمه الحواسيب الحديثه متعدده الاغراض هي شامله للحواسيب الشخصيه والحواسيب العملاقه ولها نظام تشغيل ليشغل بقيه البرامج مثل التطبيقات البرمجيه ومن الامثله علي انظمه تشغيل الحواسيب الشخصيه ويندوز ميكروسوفت ويندوز و جنولينكس و ماك اواس و داروين نظام تشغيل داروين و ماك اواساكس و يونكس والمستوي الادني من اي نظام تشغيل هو نواه علم حاسوب نواته وهذه هي الطبقه الاولي من البرمجيات التي يتم تحميلها في الذاكره عند اقلاع النظام او بد التشغيل وتوفر النواه امكانيه الوصول الي الخدمات المركزيه الشائعه الاخري لكل برامج النظام والتطبيقات وهذه الخدمات 

In [37]:
tokens3 = operating_sys.split()

new_vector3 = model.infer_vector(tokens3)
sims3 = model.docvecs.most_similar([new_vector3]) #gives you top 10 document tags and their cosine similarity

sims3

[('SENT_300222', 0.7077183723449707),
 ('SENT_565942', 0.687654972076416),
 ('SENT_104565', 0.6870350241661072),
 ('SENT_381872', 0.6841662526130676),
 ('SENT_64008', 0.6837232112884521),
 ('SENT_134521', 0.6827564835548401),
 ('SENT_227534', 0.6821423768997192),
 ('SENT_48259', 0.6808664798736572),
 ('SENT_820064', 0.6795332431793213),
 ('SENT_585438', 0.6784778237342834)]

In [38]:
get_titles(sims3,articles)

['شبكات استشعار لاسلكية',
 'خصوصية',
 'حاسوب شخصي',
 'لغة التجميع',
 'نواة (نظم تشغيل)',
 'تلوث الهواء',
 'خدمة وكيلة',
 'محرك احتراق داخلي',
 'سيارة إسعاف',
 'ويكيبيديا:الميدان/أرشيف/سياسات/06/2009']

In [39]:
get_texts(sims3,articles)

[' شبكات الاستشعار اللاسلكيه هي عباره عن مجموعه من اجهزه استشعار الاستشعار التي تستخدم في نقل او متابعه ظاهره فيزيائه او كميائيه محدده كالحراره الرطوبه الاهتزاز الضو الخ ومن ثم نقل المعلومات عن الظاهره لاسلكيا الي مركز معالجه البيانات للاستفاده منها دون الحاجه لتواجد الانسان في مكان الظاهره الفيزيائيه نظره عامه وقد بدات شبكه الاستشعار اللاسلكيه باعتبارها واحده من افضل التكنولوجيات الناشئه في القرن الحادي والعشرين تتطور بوتيره متسارعه في السنوات العشر الماضيه وقد تم القيام بالكثير من الابحاث لتحسينها في مختلف الجوانب بما في ذلك بنائها وانظمه تشغيل العقد وبروتوكولات التوجيه وجمع البيانات ودمجها واليه تحديد الموقع ومزامنه الوقت وما الي ذلك وعلاوه علي ذلك ظهرت اعداد كبيره من التطبيقات الواعده وتم نشرها في مناطق جغرافيه مختلفه مثل حمايه الهياكل الاساسيه والاستكشاف العلمي والمراقبه العسكريه ومراقبه حركه المرور والسيطره وامن التعدين والنقل البحري وحمايه البيئه وتتبع الاجسام والشؤون العسكريه وما الي ذلك مع وسائل الراحه التي قدمتها شبكه الاستشعار اللاسلكيه وقد اثرت في حياتنا وتغيرت الي حد كبير 

### example 4

### This is an article that talks about web pages.

In [42]:
web_article = articles[articles['id'] == 801146]['text'][146749]

In [43]:
web_article

'  الجداول التاليه تقارن المعلومات العامه والتقنيه لعدد من متصفحات شبكه الانترنت الرجا الاطلاع علي مقاله المنتجات الفرديه لمزيد من المعلومات تاريخ متصفح الويب الانترنت متصفحات الويب حسب السنه هذا جدول لمتصفحات الويب للكومبيوتر الشخصي بالنسبه لسنه اصدار والنسخه الرئيسيه بالترتيب الزمني مع العدد التقريبي لمستخدمي الانترنت في العالم بالملاين لاحظ بان بيانات مستخدمي الانترنت متعلقه ومتصله بالسوق باكمله ليس من الاصدرات المطلقه في تلك السنه النمو المتزايد للانترنت في التسعينات و يعني بان المتصفحات الحاليه ذات حصص سوق صغيره لها اكثر مجموع من المستخدمين في السوق باكمله علي سبيل المثال من حصص السوق في ما يقارب مليون مستخدم ولكن بحلول بدايه عام من حصص السوق يعادل اكثر من مليون مستخدم مرجع ويب المسار العنوان التاريخ الناشر السنه متصفحات الويب مستخدمي الانترنت بالمليون الشبكه العالميه ورلد وايد ويب فايولا دبليو دبليو دبليو ايروايز ميداس ماك دبليو دبليو دبليو موزايك سيللو لنكس ارينا ايه موزايك متصفح اكسبلورر اي بي ام نتسكيب نافيجيتور سليب نوت ماك ويب متصفح اي ارجو مينيت انترنت اكسبلورر نتسكيب نافيج

In [44]:
tokens4 = web_article.split()

new_vector4 = model.infer_vector(tokens4)
sims4 = model.docvecs.most_similar([new_vector4]) #gives you top 10 document tags and their cosine similarity

sims4

[('SENT_801146', 0.4568641781806946),
 ('SENT_415291', 0.4206191301345825),
 ('SENT_423217', 0.415643572807312),
 ('SENT_629628', 0.4147266149520874),
 ('SENT_704314', 0.4122125506401062),
 ('SENT_670454', 0.4106632173061371),
 ('SENT_773338', 0.4094352722167969),
 ('SENT_475328', 0.4091026782989502),
 ('SENT_456390', 0.4089905023574829),
 ('SENT_405896', 0.40890198945999146)]

In [45]:
get_titles(sims4,articles)

['مقارنة بين متصفحات الوب',
 'أوبن أوفيس.أورج بيس',
 'لعبة إيقاعية',
 'جانغو (إطار عمل ويب)',
 'آب ستور',
 'ساراواك',
 'قائمة الدول حسب مساحة الغابات',
 'إنيل',
 'قائمة السفراء من المملكة المتحدة إلى الجزائر',
 'قائمة الشركات التي استحوذت عليها ألفابت']

In [46]:
get_texts(sims4,articles)

['  الجداول التاليه تقارن المعلومات العامه والتقنيه لعدد من متصفحات شبكه الانترنت الرجا الاطلاع علي مقاله المنتجات الفرديه لمزيد من المعلومات تاريخ متصفح الويب الانترنت متصفحات الويب حسب السنه هذا جدول لمتصفحات الويب للكومبيوتر الشخصي بالنسبه لسنه اصدار والنسخه الرئيسيه بالترتيب الزمني مع العدد التقريبي لمستخدمي الانترنت في العالم بالملاين لاحظ بان بيانات مستخدمي الانترنت متعلقه ومتصله بالسوق باكمله ليس من الاصدرات المطلقه في تلك السنه النمو المتزايد للانترنت في التسعينات و يعني بان المتصفحات الحاليه ذات حصص سوق صغيره لها اكثر مجموع من المستخدمين في السوق باكمله علي سبيل المثال من حصص السوق في ما يقارب مليون مستخدم ولكن بحلول بدايه عام من حصص السوق يعادل اكثر من مليون مستخدم مرجع ويب المسار العنوان التاريخ الناشر السنه متصفحات الويب مستخدمي الانترنت بالمليون الشبكه العالميه ورلد وايد ويب فايولا دبليو دبليو دبليو ايروايز ميداس ماك دبليو دبليو دبليو موزايك سيللو لنكس ارينا ايه موزايك متصفح اكسبلورر اي بي ام نتسكيب نافيجيتور سليب نوت ماك ويب متصفح اي ارجو مينيت انترنت اكسبلورر نتسكيب نافي

### example 5

### This is an article that discusses air pollution.

In [49]:
air_pollution_article = articles[articles['id'] == 134521]['text'][29566]

In [50]:
air_pollution_article

' اعاده كتابه طقس تلوث الهوا هو تعرض الغلاف الجوي لكوكب الارض الغلاف الجوي ماده كيماويه لمواد كيماويه او الجسيمات الماديه جسيمات ماديه او المواد البيولوجيه مركبات بيولوجيه تسبب الضرر والاذي للانسان والكائنات الحيه الاخري او تؤدي الي الاضرار البيئه الطبيعيه بالبيئه الطبيعيه ملوثات الهوا يعرف ملوث الهوا بانه اي ماده في الهوا يمكن ان تسبب الضرر للانسان والبيئه ومن الممكن ان تكون هذه الملوثات في شكل جزيئات صلبه او قطرات سائله او غازات هذا بالاضافه الي انها قد تكون طبيعيه او ناتجه عن نشاط الانسان بحيث تبلغ نسبته في الوطن العربي ويمكن تصنيف الملوثات الي ملوثات اوليه وملوثات ثانويه وعاده ما تكون الملوثات الاوليه هي المواد التي تصدر بشكل مباشر من احدي العمليات مثل الرماد المتناثر من ثوره احد البراكين او غاز اول اكسيد الكربون المنبعث من عوادم السيارات او ثاني اكسيد الكربون المنبعث من مداخن المصانع اما الملوثات الثانويه فهي التي لا تنبعث في الهوا بشكل مباشر وانما تتكون هذه الملوثات في الهوا عندما تنشط الملوثات الاوليه او تتفاعل مع بعضها البعض ومن الامثله المهمه علي الملوثات الثانويه اقتراب الاوز

In [51]:
tokens5 = air_pollution_article.split()

new_vector5 = model.infer_vector(tokens5)
sims5 = model.docvecs.most_similar([new_vector5]) #gives you top 10 document tags and their cosine similarity

sims5

[('SENT_645007', 0.6555761694908142),
 ('SENT_300222', 0.6545317769050598),
 ('SENT_134521', 0.6461168527603149),
 ('SENT_301167', 0.6458536386489868),
 ('SENT_751207', 0.6442256569862366),
 ('SENT_112193', 0.6376132965087891),
 ('SENT_7', 0.6369157433509827),
 ('SENT_290741', 0.6367028951644897),
 ('SENT_88723', 0.6366755366325378),
 ('SENT_97114', 0.6360843181610107)]

In [52]:
get_titles(sims5,articles)

['تسمم بالرصاص',
 'شبكات استشعار لاسلكية',
 'تلوث الهواء',
 'اضطراب نقص الانتباه مع فرط النشاط',
 'تدخين سلبي',
 'دب قطبي',
 'ماء',
 'سوء التغذية',
 'سرطان الثدي',
 'سمنة']

In [53]:
get_texts(sims5,articles)

[' معلومات مرض صوره قبالامراض تدا تدا تدا تدا تداا وراثه مندليه بشريه مدلاين بلس ماده اي ميديسين موضوع اي ميديسين معرف نفمط معرف مراجعه جين اسم مراجعه جين التسمم رصاص بالرصاص هو نوع من انواع التسمم معدن المعدني الناجم عن تراكم رصاص الرصاص في الجسم يعتبر دماغ الدماغ هو اكثر الاعضا حساسيه للرصاص مرجع ويب العنوان المسار الموقع تاريخ الوصول التاريخ قد تشمل اعراض التسمم الم بطني الام في البطن امساك دا والامساك صداع والصداع تهيجيه والتهيج ومشاكل الذاكره وعدم القدره علي انجاب الاطفال عقم العقم وخز في اليدين والقدمين يسبب التسمم بالرصاص ما يقرب من من حالات تخلف عقلي التخلف العقلي التي تحدث لاسباب غير معروفه وخلافا لذلك ويمكن ان يؤدي ذلك الي مشاكل سلوكيه يخلف تسمم الرصاص بعض الاثار الدائمه مرجع ويب العنوان المسار الموقع تاريخ الوصول التاريخ وقد يحدث في الحالات الشديده فقر الدم انيميا فقر الدم نوبه طب والنوبات غيبوبه والغيبوبه او الوفاه يمكن ان يتعرض المر للرصاص عن طريق الهوا الملوث او الما او الغبار او الطعام او المنتجات الاستهلاكيه الاطفال معرضون لخطر اكبر لانهم اكثر عرضه لالتقاط الاشيا من الا

# Saving the model.

In [54]:
model.save("F:\\Hackathon\\easyLearn\\Wikipedia Model - one_hour_and_33_minutes_trained_model\\one_hour_and_33_minutes_trained_model")